#### import dos pacotes necessários

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

pd.set_option('display.max_rows', 500)

#### import dos dados

In [2]:
# import dos dados em um df chamado df_papa
df_papa = pd.read_csv('papafrescaSimNao.csv', sep=';')

In [3]:
# formato do df criado
df_papa.shape

(20869, 23)

In [4]:
# verificando se há alguma resposta nula
df_papa.isna().sum()

TV                   0
RADIO                0
PRENSA               0
INTERNET             0
Lapapaessaludable    0
Lapapadaenergia      0
Lapapaengorda        0
Lapapaeseconomica    0
Vapor                0
Sopa                 0
Pure                 0
Fritas               0
Sudada               0
Salada               0
OtrasFormas          0
SOPAS                0
GRANOS               0
CARNE                0
HUEVOS               0
ARROZ                0
ENSALADA             0
NADAONINGUNO         0
OTROS                0
dtype: int64

O conjunto de dados contém cerca de 21k entradas, e 23 colunas

In [5]:
# nome das colunas
df_papa.columns

Index(['TV', 'RADIO', 'PRENSA', 'INTERNET', 'Lapapaessaludable',
       'Lapapadaenergia', 'Lapapaengorda', 'Lapapaeseconomica', 'Vapor',
       'Sopa', 'Pure', 'Fritas', 'Sudada', 'Salada', 'OtrasFormas', 'SOPAS',
       'GRANOS', 'CARNE', 'HUEVOS', 'ARROZ', 'ENSALADA', 'NADAONINGUNO',
       'OTROS'],
      dtype='object')

In [6]:
# view dos dados importados
df_papa.head()

,TV,RADIO,PRENSA,INTERNET,Lapapaessaludable,Lapapadaenergia,Lapapaengorda,Lapapaeseconomica,Vapor,Sopa,...,Salada,OtrasFormas,SOPAS,GRANOS,CARNE,HUEVOS,ARROZ,ENSALADA,NADAONINGUNO,OTROS
0,Nao,Nao,Nao,Nao,Nao,Nao,Sim,Sim,Nao,Nao,...,Nao,Nao,Nao,Nao,Sim,Nao,Sim,Nao,Nao,Nao
1,Nao,Nao,Nao,Nao,Nao,Nao,Sim,Sim,Nao,Nao,...,Nao,Nao,Nao,Nao,Nao,Nao,Sim,Nao,Nao,Sim
2,Nao,Nao,Nao,Nao,Nao,Nao,Sim,Sim,Nao,Nao,...,Nao,Sim,Nao,Nao,Nao,Nao,Nao,Nao,Sim,Nao
3,Nao,Nao,Nao,Nao,Nao,Nao,Sim,Sim,Nao,Nao,...,Nao,Nao,Nao,Nao,Nao,Nao,Sim,Nao,Nao,Sim
4,Nao,Nao,Nao,Nao,Nao,Nao,Sim,Sim,Nao,Nao,...,Sim,Nao,Nao,Nao,Sim,Nao,Sim,Nao,Nao,Nao


In [7]:
# vamos entender as proporções de cada resposta no df
for col in df_papa.columns:
    print('\n\n ---------{}---------'.format(col))
    print(df_papa[col].value_counts(normalize=True))



 ---------TV---------
Nao    0.612679
Sim    0.387321
Name: TV, dtype: float64


 ---------RADIO---------
Nao    0.983564
Sim    0.016436
Name: RADIO, dtype: float64


 ---------PRENSA---------
Nao    0.996406
Sim    0.003594
Name: PRENSA, dtype: float64


 ---------INTERNET---------
Nao    0.976233
Sim    0.023767
Name: INTERNET, dtype: float64


 ---------Lapapaessaludable---------
Nao    0.720734
Sim    0.279266
Name: Lapapaessaludable, dtype: float64


 ---------Lapapadaenergia---------
Nao    0.87618
Sim    0.12382
Name: Lapapadaenergia, dtype: float64


 ---------Lapapaengorda---------
Nao    0.571422
Sim    0.428578
Name: Lapapaengorda, dtype: float64


 ---------Lapapaeseconomica---------
Sim    0.731803
Nao    0.268197
Name: Lapapaeseconomica, dtype: float64


 ---------Vapor---------
Nao    0.905075
Sim    0.094925
Name: Vapor, dtype: float64


 ---------Sopa---------
Nao    0.597441
Sim    0.402559
Name: Sopa, dtype: float64


 ---------Pure---------
Nao    0.932196
Sim   

- Vemos que a grande maioria das respostas é não
- Todas as colunas têm somente a informação Sim ou Não

#### Transformação dos dados

In [8]:
# como todas as respostas são Sim ou Não, podemos transformar os dados em binário no df_papa 

for col in df_papa.columns:
    df_papa[col] = df_papa[col].apply(lambda x: True if x=='Sim' else False)
    
df_papa.head()

,TV,RADIO,PRENSA,INTERNET,Lapapaessaludable,Lapapadaenergia,Lapapaengorda,Lapapaeseconomica,Vapor,Sopa,...,Salada,OtrasFormas,SOPAS,GRANOS,CARNE,HUEVOS,ARROZ,ENSALADA,NADAONINGUNO,OTROS
0,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,True,False,True,False,False,False
1,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,True,False,False,True
2,False,False,False,False,False,False,True,True,False,False,...,False,True,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,True,True,False,False,...,False,False,False,False,False,False,True,False,False,True
4,False,False,False,False,False,False,True,True,False,False,...,True,False,False,False,True,False,True,False,False,False


#### Aplicação do A Priori

Neste ponto vale destacar que um suporte muito baixo dá uma quantidade muito alta de combinações, e os *insights* obtidos não serão de grande valor, pois representam poucos casos do total. Assim, optaremos por um **suporte mínimo de 10%**.

In [9]:
# Como todos os dados do df são binários, podemos aplicar o apriori diretamente no df, sem a 
# etapa de transformação em lista de transações

itemsets_freq = apriori(df_papa, min_support=0.1, use_colnames=True)
print(itemsets_freq.sort_values(by=['support'], ascending=False))

     support                                           itemsets
4   0.731803                                (Lapapaeseconomica)
9   0.609612                                            (ARROZ)
8   0.555225                                            (CARNE)
33  0.440174                         (Lapapaeseconomica, ARROZ)
3   0.428578                                    (Lapapaengorda)
32  0.406919                         (Lapapaeseconomica, CARNE)
5   0.402559                                             (Sopa)
43  0.399300                                     (CARNE, ARROZ)
0   0.387321                                               (TV)
24  0.302458                 (Lapapaengorda, Lapapaeseconomica)
68  0.289664                  (Lapapaeseconomica, CARNE, ARROZ)
29  0.287508                          (Lapapaeseconomica, Sopa)
15  0.285974                            (Lapapaeseconomica, TV)
1   0.279266                                (Lapapaessaludable)
37  0.269107                            

#### Obtenção das Regras de Associação

In [10]:
# Obtém as regras de associação a partir dos itemsets mais frequentes. 
# Como sugerido no exercício, filtramos para os casos com confiança mínima de 50%

regras = association_rules(itemsets_freq, metric="confidence", min_threshold=0.5)


#Ordena as Regras por confiança
regrasOrdenadas = regras.sort_values('confidence' , ascending=False)

In [11]:
regrasOrdenadas.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(Lapapadaenergia),(TV),0.123820,0.387321,0.120130,0.970201,2.504903,0.072172,20.560556
32,"(Lapapaengorda, Lapapaessaludable)",(TV),0.107192,0.387321,0.103790,0.968261,2.499894,0.062272,19.303706
40,"(Lapapaessaludable, CARNE)",(TV),0.156979,0.387321,0.150031,0.955739,2.467563,0.089230,13.842322
38,"(Lapapaessaludable, Sopa)",(TV),0.113757,0.387321,0.108103,0.950295,2.453508,0.064042,12.326273
44,"(Lapapaessaludable, ARROZ)",(TV),0.160142,0.387321,0.151756,0.947636,2.446643,0.089730,11.700420


In [12]:
# mantendo somente as colunas de interesse
regrasOrden = regrasOrdenadas[['antecedents', 'consequents', 'support', 'confidence']]

In [13]:
regrasOrden.head()

,antecedents,consequents,support,confidence
2,(Lapapadaenergia),(TV),0.120130,0.970201
32,"(Lapapaengorda, Lapapaessaludable)",(TV),0.103790,0.968261
40,"(Lapapaessaludable, CARNE)",(TV),0.150031,0.955739
38,"(Lapapaessaludable, Sopa)",(TV),0.108103,0.950295
44,"(Lapapaessaludable, ARROZ)",(TV),0.151756,0.947636


## Análise das Regras de Associação

Como vimos anteriormente, os casos em 'Sim' são muito mais raros e, portanto, os casos de maior interesse.
Portanto, vamos focar a análise nas regras relacionadas ao questionamento de quando a batata engorda. 

**Vamos portanto entender: o que ocorre quando quando as pessoas acreditam que a batata engorda?**

In [25]:
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrden.iterrows():
    for ent in row['antecedents']:
        if ent == 'Lapapaengorda':
            filtro.append(index)
            break

In [26]:
regrasOrden.loc[regrasOrden.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support
32,"(Lapapaengorda, Lapapaessaludable)",(TV),0.968261,0.103790
69,"(Lapapaengorda, CARNE)",(ARROZ),0.741553,0.176674
106,"(Lapapaengorda, Lapapaeseconomica, CARNE)",(ARROZ),0.728872,0.121089
48,"(Lapapaengorda, TV)",(Lapapaeseconomica),0.718979,0.108007
9,(Lapapaengorda),(Lapapaeseconomica),0.705725,0.302458
66,"(Lapapaengorda, ARROZ)",(Lapapaeseconomica),0.699784,0.186640
62,"(Lapapaengorda, Sopa)",(Lapapaeseconomica),0.697755,0.125114
64,"(Lapapaengorda, CARNE)",(Lapapaeseconomica),0.697305,0.166132
33,"(Lapapaengorda, TV)",(Lapapaessaludable),0.690909,0.103790
108,"(Lapapaengorda, CARNE, ARROZ)",(Lapapaeseconomica),0.685381,0.121089


- O primeiro *insight* otido é que 97% pessoas que pensam que a batata engorda e é saudável viram propaganda na TV
- Um outro ponto interessasnte é sobre o consumo: 74% das pessoas que pensam que a batata engorda e a comem com carne usam o acompanhamento arroz, assim como as pessoas que pensam que engorda, que é econômica e a comem com carne (73%). Se mirarmos no caso exclusivo das pessoas que pensam que a batata engorda, 56% a usam com carne como acompanhamento
- Também é interessante notar que 69% das pessoas que pensam que a batata engorda E que viram propaganda na TV pensam que a batata é saudável

**Vamos agora entender a outra situação: quais casos normalmente ocorrem que levam as pessoas a pensar que a batata engorda?**

In [20]:
# criando lista com índices de interesse
filtro = []
for index, row in regrasOrden.iterrows():
    for ent in row['consequents']:
        if ent == 'Lapapaengorda':
            filtro.append(index)
            break

In [21]:
regrasOrden.loc[regrasOrden.index.isin(filtro), : ][['antecedents','consequents','confidence', 'support']]

,antecedents,consequents,confidence,support


- Vemos que não há nenhum caso de interesse nesta situação, por um dos dois fatores: ou a frequência mínima não atingiu 10% dos casos ou o suporte não chegou a 50%. Em ambos os casos, de fato não há itens de interesse, e os principais *insights* estão na análise anterior!